# Data Science Challenge - Part 2: State Spending <br> (by Peter Kretschmer)

The notebook is structured as follows:
1. Exploratory Data Analysis
2. Data preprocessing
3. Solutions to the questions

## 1. Exploratory Data Analysis

First we import packages needed later on.

In [1]:
import pandas as pd
import numpy as np
import re
import os

Next, we read in the csv file.

In [2]:
file_name ='res_purchase_2014.csv'

In [3]:
file_path = os.path.abspath(file_name)

In [4]:
df = pd.read_csv(file_path,dtype={'Amount':object})

When we do not specify the dtype of column 'Amount', this will lead to a warning that the corresponding column has mixed types. This already indicates us that we should be aware of that column and might need some preprocessing.

We continue and have a first look at the data, it's shape and the column types.

In [5]:
df.head(5)

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC)
0,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,GENERAL PURCHASE,890,NACAS,7/30/2013 0:00,7/31/2013 0:00,CHARITABLE AND SOCIAL SERVICE ORGANIZATIONS
1,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,ROOM CHARGES,368.96,SHERATON HOTEL,7/30/2013 0:00,7/31/2013 0:00,SHERATON
2,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,J,GENERAL PURCHASE,165.82,SEARS.COM 9300,7/29/2013 0:00,7/31/2013 0:00,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...
3,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,T,GENERAL PURCHASE,96.39,WAL-MART #0137,7/30/2013 0:00,7/31/2013 0:00,"GROCERY STORES,AND SUPERMARKETS"
4,201307,1000,OKLAHOMA STATE UNIVERSITY,Mauro-Herrera,M,HAMMERMILL COPY PLUS COPY EA,125.96,STAPLES DIRECT,7/30/2013 0:00,7/31/2013 0:00,"STATIONERY, OFFICE SUPPLIES, PRINTING AND WRIT..."


We see that the column 'Amount' should contain numerical values. We keep that in mind for the preprocessing.

In [6]:
print('Number of rows: {} \nNumber of columns: {}'.format(df.shape[0],df.shape[1]))

Number of rows: 442457 
Number of columns: 11


In [7]:
df.dtypes

Year-Month                       int64
Agency Number                    int64
Agency Name                     object
Cardholder Last Name            object
Cardholder First Initial        object
Description                     object
Amount                          object
Vendor                          object
Transaction Date                object
Posted Date                     object
Merchant Category Code (MCC)    object
dtype: object

We check for NA's.

In [8]:
df.isnull().sum().sum()

0

There are none. That's nice!

## 2. Data preprocessing

Basically,we want to
- preprocess column 'Amount' for Question 1
- add 'Year' and 'Month' Column for Question 4

So first we take a look at the amount column:

In [9]:
df['Amount'].iloc[5:30]

5          394.28
6           19.87
7            10.1
8           142.6
9        ($29.99)
10         278.23
11          127.8
12       $572.27 
13        $12.90 
14            445
15    452.91 zero
16           2425
17           2425
18           7.63
19          18.97
20          729.8
21          729.8
22            368
23           9.16
24           1.68
25            -30
26            178
27          27.44
28          54.72
29          112.8
Name: Amount, dtype: object

We see that the column contains several values in a wrong format including dollar signs, brackets, additional text and space characters. Moreover, there are also negative values. After having a look at the data, we decide to transform them to positive values as they capture purchases (and not paybacks for example) and should therefore be positive. Hence, we will define a regular expression to detect the floats in the string and transform the column.

In [10]:
regex_amount = re.compile(r"[+]?\d*\.\d+|\d+") 

In [11]:
df['Amount'] = df['Amount'].astype(str).apply(lambda x: regex_amount.search(x)[0]).astype(float)

Re-checking the data, we see that everything looks fine now.

We now turn to preprocessing step 2. For Question 4 we have to calculate monthly spendings. Therefore, we want to introduce a new 'Year' and 'Month' column. So, lets first have a look at the corresponding values of the 'Date' column.

In [12]:
df['Year-Month'].unique()

array([201307, 201402, 201401, 201406, 201309, 201403, 201308, 201310,
       201311, 201312, 201405, 201404, 201900,   -999], dtype=int64)

Obviously, we have information for date '201900' and '-999'. There must have been an error for those values, so we construct the Year and Month column using the column 'Posted Date' as this coincides with the 'Year-Month' column in their year-month information for all 'valid' years (not 201900 and -999). We will also re-check in a few lines that this is indeed the case.

So again we define regular expression to get the year and month column from the 'Posted Date' column.

In [13]:
regex_year = re.compile(r"\d{4}")

In [14]:
df['Year'] = df['Posted Date'].apply(lambda x: regex_year.search(x)[0]).astype(str)

In [15]:
regex_month = re.compile(r"\d{1,2}")

In [16]:
df['Month'] = df['Posted Date'].apply(lambda x: regex_month.match(x)[0]).astype(str).apply(lambda x: x.zfill(2))

We will quickly re-check that indeed for all the 'valid' values in the 'Year-Month' column, the year month information coincides with the information from the 'Posted Date' column.

In [17]:
valid_years = df['Year-Month'].unique()[:12]

In [18]:
year_mask = df['Year-Month'].isin(valid_years)

In [19]:
all(df['Year-Month'].astype(str)[year_mask] == (df['Year'] + df['Month'])[year_mask])

True

Nice, let's quickly see if we now have reasonable 'Year' and 'Month' information.

In [20]:
df['Year'].unique()

array(['2013', '2014'], dtype=object)

In [21]:
df['Month'].unique()

array(['07', '02', '01', '06', '09', '03', '08', '10', '11', '12', '05',
       '04'], dtype=object)

That was the preprocessing needed for the questions to answer!

## 3. Solutions to the questions

**Q1: What is the total amount of spending captured in this dataset? **

In [22]:
print('The total amount of spending is {}'.format(df['Amount'].sum()))

The total amount of spending is 195165873.33


**Q2: How much was spent at WW GRAINGER? **

For Question 2 and 3 we want to be sure that we catch all transactions for the corresponding vendor, i.e. all branches and/or subcontractors. Hence, we define a Pandas Series that gives us all unique vendor names.

In [23]:
Vendor_unique_values = pd.Series(df['Vendor'].unique())

We define a regular expression that will give us all vendors with 'WW GRAINGER' in the name.

In [24]:
def apply_regex_vendor(regex,name,vendor_list):
    x = regex.search(name)
    if (x != None):
        vendor_list.append(x.string)

In [25]:
regex_grainger = re.compile(r"WW GRAINGER")
vendor_WW_GRAINGER = list()
Vendor_unique_values.apply(lambda name: apply_regex_vendor(regex_grainger,name,vendor_WW_GRAINGER));

In [26]:
print(vendor_WW_GRAINGER)

['WW GRAINGER', 'Claim ADJ/WW GRAINGER']


Indeed, as we expected, we get one more vendor to look at. So we get the result:

In [27]:
print('The total amount spent at WW GRAINGER is {}'.format(df[df['Vendor'].isin(vendor_WW_GRAINGER)]['Amount'].sum()))

The total amount spent at WW GRAINGER is 5360849.3


**Q3: How much was spent at WM SUPERCENTER? **

For the vendor 'WM SUPERCENTER' this is then analogoue.

In [28]:
regex_supercenter = re.compile(r"WM SUPERCENTER")
vendor_WM_SUPERCENTER = list()
Vendor_unique_values.apply(lambda name: apply_regex_vendor(regex_supercenter,name,vendor_WM_SUPERCENTER));

In [29]:
print(vendor_WM_SUPERCENTER[:5])

['WM SUPERCENTER', 'WM SUPERCENTER #121', 'WM SUPERCENTER #38', 'WM SUPERCENTER #479', 'WM SUPERCENTER #41']


Indeed we also have here more than vendor corresponding to WM SUPERCENTER.

In [30]:
print('The total amount spent at WM SUPERCENTER is {}'.format(df[df['Vendor'].isin(vendor_WM_SUPERCENTER)]['Amount'].sum()))

The total amount spent at WM SUPERCENTER is 157457.46


**Q4: What is the standard deviation of the total monthly spending in the dataset? **

In [31]:
print('The standard deviation of the total monthly spending is {}'.format(df.groupby('Month')['Amount'].sum().std()))

The standard deviation of the total monthly spending is 2076717.1945410317
